In [83]:
from Crypto.Cipher import AES
from base64 import b64encode
from Crypto.Util.Padding import pad, unpad
import base64
import numpy as np

In [86]:
default_iv = b"\x00" * 16

def customized_pad(bmessage, block_len):
    pad_value = block_len-1 - (len(bmessage)-1) % block_len
    assert block_len < 256
    pad_bytes = bytes.fromhex("{:02x}".format(pad_value)) * pad_value
    return bmessage + pad_bytes

def encrypt_CBC(bmessage, key, iv=default_iv, block_len=16):
    assert len(key) == block_len
    pbmessage = customized_pad(bmessage, block_len)
    n_blocks = len(pbmessage) // block_len
    emessage_block = iv
    emessage = b""
    cipher = AES.new(key, AES.MODE_ECB)
    for i in range(n_blocks):
        xormessage = bytes([c1^c2 for c1, c2 in zip(pbmessage[i*block_len:(i+1)*block_len], emessage_block)])
        emessage_block = cipher.encrypt(xormessage)
        emessage += emessage_block

    return emessage

def decrypt_CBC(emessage, key, iv=default_iv, block_len=16):
    assert len(key) == block_len
    assert len(emessage) % block_len == 0
    n_blocks = len(emessage) // block_len
    dmessage_block = iv
    cipher = AES.new(key, AES.MODE_ECB)
    original_message = b""
    for i in range(n_blocks):
        emessage_block = emessage[i*block_len:(i+1)*block_len]
        xor_message_block = cipher.decrypt(emessage_block) 
        original_message_block = bytes([c1^c2 for c1, c2 in zip(xor_message_block, dmessage_block)])
        original_message += original_message_block
        dmessage_block = emessage_block
    return original_message

def load(url):
    raw_message = np.loadtxt(url, dtype="str")
    return raw_message

def base64decode(raw_message):  
    message = base64.b64decode(raw_message)
    return message

In [ ]:
bm = b"abcdefghijklmnopqrstuvwxyz"
key = b"YELLOW SUBMARINE"
block_len = 16
encrypted_message = encrypt_CBC(bmessage=bm, key=key, iv=default_iv, block_len=block_len)

In [81]:
unpad(decrypt_CBC(emessage=encrypted_message, key=key, iv=default_iv, block_len=block_len), block_len)

b'abcdefghijklmnopqrstuvwxyz'

In [91]:
url = "https://cryptopals.com/static/challenge-data/10.txt"
message = base64decode(load(url=url))

print(unpad(decrypt_CBC(message, key=b"YELLOW SUBMARINE"), block_len).decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 
